In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tmdbsimple as tmdb
import os, time,json
from tqdm.notebook import tqdm_notebook

In [2]:
# loading previous MovieTime Analysis notebook dataframes
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

In [3]:
# checking for missing values
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82150 entries, 0 to 82149
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82150 non-null  object 
 1   titleType       82150 non-null  object 
 2   primaryTitle    82150 non-null  object 
 3   originalTitle   82150 non-null  object 
 4   isAdult         82150 non-null  int64  
 5   startYear       82150 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82150 non-null  int64  
 8   genres          82150 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.6+ MB


In [4]:
# dropping endYear column, all null
basics = basics.drop(columns = ["endYear"])

In [5]:
# checking for missing values
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342956 entries, 0 to 1342955
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1342956 non-null  object 
 1   ordering         1342956 non-null  int64  
 2   title            1342956 non-null  object 
 3   region           1342956 non-null  object 
 4   language         3681 non-null     object 
 5   types            963321 non-null   object 
 6   attributes       44751 non-null    object 
 7   isOriginalTitle  1341581 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 82.0+ MB


In [6]:
# replacing null values as unknown
akas["language"].fillna("Unknown", inplace = True)
akas["types"].fillna("Unknown", inplace = True)
akas["attributes"].fillna("Unknown", inplace = True)
akas["isOriginalTitle"].fillna("Unknown", inplace = True)

In [7]:
# check for missing values
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263582 entries, 0 to 1263581
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1263582 non-null  object 
 1   averageRating  1263582 non-null  float64
 2   numVotes       1263582 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB


In [8]:
# tmdbsimple
!pip install tmdbsimple  --no-warn-script-location

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# API credentials
with open('/Users/joshu/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [10]:
# instantiating API key
tmdb.API_KEY =  login['api-key']

In [11]:
# designating folder 'Data' to save my API calls
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [12]:
# my client's desired years
YEARS_TO_GET = [2000, 2001]

In [13]:
# helper functions prior to running loop
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## if read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [14]:
def movie_rating(movie_id):
    
    # getting the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # save the .info .releases dictionaries
    movie_info = movie.info()
    
    releases = movie.releases()
    # looping through countries in releases
    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
           movie_info['certification'] = c['certification']
    return movie_info

In [15]:
# preparing the inner loop  (https://www.geeksforgeeks.org/append-to-json-file-using-python/)
def write_json(new_data, filename):   
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [16]:
# outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = read_and_fix_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]        

    # inner loop
    for movie_id in tqdm_notebook(movie_ids_to_get, 
                                  desc=f'Movies from {YEAR}', 
                                  position=1, leave=True):
        try:
            temp = movie_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
            
        except Exception as e:
            continue
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1407 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1529 [00:00<?, ?it/s]